In [2]:
# check which GPU is used
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
# uploading helper function
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-10-22 19:31:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-10-22 19:31:58 (83.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# using tensorflow datasets to download the data
import tensorflow_datasets as tfds

In [5]:
# get all datasets and find "food101"
data_list = tfds.list_builders()
print("food101"in data_list)

True


In [6]:
# load food101 data set
(train_data, test_data), ds_info= tfds.load(name = "food101",
                                            split=["train","validation"],
                                            with_info = True,
                                            as_supervised= True,
                                            shuffle_files= True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: ignored

In [ ]:

#inspect the data
ds_info

In [ ]:
class_names = ds_info.features["label"].names

In [ ]:

for image , label in train_data.take(1):
  print(f""" 
  image shape {image.shape}
  image data type {image.dtype}
  image label { label}
  image class {class_names[label.numpy()]}

  """)
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)



In [ ]:
# vilualise more 
import matplotlib.pyplot as plt


In [ ]:
len(train_data)

# Preprocessing our data

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
def preprocess_img(image, label, image_shape=None):
  resized_image = tf.image.resize(image, (224,224))
  return tf.cast(resized_image, tf.float32) ,label  #change data type

# Batch and Prepare your dataset

In [ ]:
# maping the data
train_data_2 = train_data.map(map_func= preprocess_img, num_parallel_calls= tf.data.AUTOTUNE)
train_data_2 = train_data_2.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)


test_data_2 = test_data.map(map_func= preprocess_img, num_parallel_calls= tf.data.AUTOTUNE)
test_data_2 = test_data_2.batch(32).prefetch(tf.data.AUTOTUNE)

# creating calbacks and mixedprecition

In [ ]:
from helper_functions import create_tensorboard_callback

In [ ]:
check_point = tf.keras.callbacks.ModelCheckpoint("check/cp.ckpt",
                                   monitor="val_acc",
                                   save_best_only=True,
                                   save_weights_only=True)

In [ ]:
tensorboard = create_tensorboard_callback("tensorboard", "faeturextrator")

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("float16")

# create baseline 

In [ ]:
import tensorflow as tf

In [ ]:
baseline = tf.keras.applications.EfficientNetB0(include_top=False)
baseline.trainable = False # freeze model
inputs  = layers.Input((224,224,3))
x = baseline(inputs, training = False)
x = layers.GlobalAvgPool2D()(x)
x = layers.Dense(len(class_names)) (x)
outputs = layers.Activation("softmax", dtype= tf.float32)  (x)
model_1 = tf.keras.Model(inputs, outputs)

In [ ]:
model_1.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = "accuracy")

In [ ]:
for layer in model_1.layers:
  print(layer.dtype, layer.dtype_policy)

In [ ]:
history_1 = model_1.fit(train_data_2, epochs = 5,
                        steps_per_epoch =len(train_data_2),
                        validation_data= test_data_2,
                        validation_steps= len(test_data_2),
                        callbacks=[check_point, tensorboard])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_1.evaluate(test_data_2)

In [ ]:
model_1.save("/content/drive/MyDrive/Colab Notebooks/food101")

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip 


In [ ]:
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

In [ ]:
model_1_loaded = tf.keras.models.load_model("/content/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
model_1_loaded.evaluate(test_data_2)

In [ ]:
model_1_loaded.layers[1].trainable = True

In [ ]:
for num, layer in enumerate(model_1_loaded.layers[1].layers):
  print(f"""
  {num} : {layer.name} , {layer.trainable} , {layer.dtype} ,{layer.dtype_policy}
  """)

In [ ]:
model_1_loaded.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = "accuracy")

In [ ]:
es=tf.keras.callbacks.EarlyStopping("val_acc" )

In [ ]:
check_point_finetued = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/check/cp.ckpt", monitor="val_acc",
                                   save_best_only=True,
                                   save_weights_only=True
                                                          )

In [ ]:
history_1_loaded = model_1_loaded.fit(train_data_2, epochs = 100,
                        steps_per_epoch =len(train_data_2),
                        validation_data= test_data_2,
                        validation_steps= 0.25*len(test_data_2),
                        callbacks=[check_point_finetued,es, tensorboard],
                        initial_epoch=5)

In [ ]:
import tensorflow as tf

In [ ]:
model_1_loaded = tf.keras.models.load_model("/content/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
model_1_loaded.load_weights("")

In [ ]:
model_1_loaded.evaluate(test_data_2)

# 2- RNN  model